# Generative AI Day 1

## Import required libraries

In [ ]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

## About the imports


  - **os**: Provides a portable way to interact with the operating system.
    
  - **requests**: Simplifies making HTTP web requests.
  - **load_dotenv**: Loads environment variables from a .env file into Python application's environment.
  - **BeautifulSoup**: Library used for parsing HTML and XML documents.
  - **Markdown**: Used to render Markdown-formatted text in a visually styled format.
  - **display**: A generic function that renders objects in appropriate formats.
  - **OpenAI**: Allows to interact with OpenAI's AI models

## Connecting to OpenAI

### Load the secret key in current Python environment

In [ ]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found but seems it is not starting with 'sk-proj-'")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end")
else:
    print("API key found and looks good so far")
    

### Create object of OpenAI class using `OpenAI()` function

In [ ]:
openai = OpenAI()

### Let's make a quick call to a Frontier model to get started, as a preview!

In [ ]:
message = "Hello, GPT! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role": "user", "content": message}])
print(response.choices[0].message.content)

## Let's start with our first project

In [ ]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    def __init__(self, url):
        """
            Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [ ]:
pmo = Website("https://www.pmindia.gov.in/en/message-from-the-prime-minister/")
print(pmo.title)
print(pmo.text)

### Types of prompts
- **A user prompt**: the conversation starter that LLM should reply to.
  
- **A system prompt**: this tells them what task LLM are performing and what tone LLM should use.

In [ ]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [ ]:
# Function that writes a User prompt
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled{website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [ ]:
print(user_prompt_for(pmo))

### Messages
The API from OpenAI expects to receive messages in a particular structure. Many of the other API's share this structure:

`[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
    ]`

So, now lets build useful messages for GPT-4o-mini, using a function.

In [ ]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [ ]:
messages_for(pmo)

### Now let's bring it all together

In [ ]:
def summerize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages_for(website)
    )
    return response.choices[0].message.content 

In [ ]:
summerize("https://www.pmindia.gov.in/en/message-from-the-prime-minister/")

In [ ]:
def display_summary(url):
    summary = summerize(url)
    display(Markdown(summary))

In [ ]:
display_summary("https://www.pmindia.gov.in/en/message-from-the-prime-minister/")